In [ ]:
import polars as pl
from math import sqrt
import queue
import random
import rtsvg
rt = rtsvg.RACETrack()
import os
import kagglehub
path = kagglehub.dataset_download("konivat/tree-of-life")
dfs = []
for _filename_ in os.listdir(path):
    df = pl.read_csv(os.path.join(path, _filename_))
    dfs.append(df)

In [ ]:
import copy
#
# CirclePacker()
# 
# Implementation of the following paper:
#
#@inproceedings{10.1145/1124772.1124851,
#               author = {Wang, Weixin and Wang, Hui and Dai, Guozhong and Wang, Hongan},
#               title = {Visualization of large hierarchical data by circle packing},
#               year = {2006},
#               isbn = {1595933727},
#               publisher = {Association for Computing Machinery},
#               address = {New York, NY, USA},
#               url = {https://doi.org/10.1145/1124772.1124851},
#               doi = {10.1145/1124772.1124851},
#               abstract = {In this paper a novel approach is described for tree visualization using nested circles. 
#                           The brother nodes at the same level are represented by externally tangent circles; 
#                           the tree nodes at different levels are displayed by using 2D nested circles or 3D nested cylinders. 
#                           A new layout algorithm for tree structure is described. It provides a good overview for large data sets. 
#                           It is easy to see all the branches and leaves of the tree. The new method has been applied to the 
#                           visualization of file systems.},
#               booktitle = {Proceedings of the SIGCHI Conference on Human Factors in Computing Systems},
#               pages = {517–520},
#               numpages = {4},
#               keywords = {tree visualization, nested circles, file system, circle packing},
#               location = {Montr\'{e}al, Qu\'{e}bec, Canada},
#               series = {CHI '06} }
#
class CirclePacker(object):
    #
    # __init__()
    #
    def __init__(self, rt_self, circles, epsilon=0.01):
        self.rt_self      = rt_self
        self.circles      = circles
        self.circles_left = copy.deepcopy(circles)
        self.epsilon      = epsilon
        self.packed       = []
        self.fwd          = {}
        self.bck          = {}
        self.debug_str    = ''
        # Pack the (up to) first four circles
        self.__packFirstFourCircles__()
        # Create a priority queue to find the circle w/in the chain that is closest to the origin
        self.nearest  = queue.PriorityQueue()
        for i in range(len(self.packed)):
            if i not in self.fwd: continue
            c = self.packed[i]
            self.nearest.put((c[0]**2 + c[1]**2, len(self.packed) - 1))
        # Pack the circles iteratively
        while len(self.circles_left) > 0: self.__packNextCircle__()
    
    #
    # __packFirstFourCircles__()
    #
    def __packFirstFourCircles__(self):
        # Circle 1
        cx0, cy0, r0  = self.circles_left.pop(0)
        cx0 = cy0 = 0.0
        self.packed.append((cx0, cy0, r0))
        if len(self.circles_left) == 0: 
            self.fwd, self.bck = {0:0}, {0:0}
            return

        # Circle 2
        cx1, cy1, r1  = self.circles_left.pop(0)
        cy1           = 0.0
        cx1           = r0 + r1
        self.packed.append((cx1, 0.0, r1))
        if len(self.circles_left) == 0: 
            self.fwd, self.bck = {0:1, 1:0}, {0:1, 1:0}
            return

        # Circle 3
        cx2, cy2, r2  = self.circles_left.pop(0)
        xy0, xy1      = self.rt_self.overlappingCirclesIntersections((cx0,cy0,r0+r2),(cx1,cy1,r1+r2))
        cx2, cy2      = xy0[0], xy0[1]
        self.packed.append((cx2, cy2, r2))
        if len(self.circles_left) == 0:
            self.fwd, self.bck = {0:1, 1:2, 2:0}, {1:0, 2:1, 0:2}
            return

        # Circle 4
        cx3, cy3, r3 = self.circles_left.pop(0)
        xy0, xy1     = self.rt_self.overlappingCirclesIntersections((cx0,cy0,r0+r3),(cx1,cy1,r1+r3))
        cx3, cy3     = xy1[0], xy1[1]
        if self.rt_self.circlesOverlap((cx2, cy2, r2), (cx3, cy3, r3)):
            xy0, xy1  = self.rt_self.overlappingCirclesIntersections((cx0,cy0,r0+r3),(cx2,cy2,r2+r3))
            cx3, cy3  = xy0[0], xy0[1]
            if self.rt_self.circlesOverlap((cx1, cy1, r1), (cx3, cy3, r3)) == False:
                self.packed.append((cx3, cy3, r3))
                self.fwd, self.bck = {0:1, 1:2, 2:3, 3:0}, {1:0, 2:1, 3:2, 0:3}
                return
            cx3, cy3  = xy1[0], xy1[1]
            if self.rt_self.circlesOverlap((cx1, cy1, r1), (cx3, cy3, r3)) == False: raise Exception("Should Never Happen (Case 2 in __packFirstFourCircles__)")
            xy0, xy1  = self.rt_self.overlappingCirclesIntersections((cx1,cy1,r1+r3),(cx2,cy2,r2+r3))
            cx3, cy3  = xy0[0], xy0[1]
            if self.rt_self.circlesOverlap((cx0, cy0, r0), (cx3, cy3, r3)) == False:
                self.packed.append((cx3, cy3, r3))
                self.fwd, self.bck = {1:2, 2:3, 3:1}, {2:1, 3:2, 1:3}
                return
            cx3, cy3  = xy1[0], xy1[1]
            if self.rt_self.circlesOverlap((cx0, cy0, r0), (cx3, cy3, r3)) == False: raise Exception("Should Never Happen (Case 4 in __packFirstFourCircles__)")
            raise Exception('How is this possible? (__packFirstFourCircles__)')
        else:
            self.packed.append((cx3, cy3, r3))
            self.fwd, self.bck = {2:0, 1:2, 3:1, 0:3}, {0:2, 2:1, 1:3, 3:0}

    #
    # _repr_svg_() - return an SVG representation
    #
    def _repr_svg_(self):
        w, h    = 250, 250
        _pkd_   = self.packed
        _chn_   = self.fwd
        _color_ = '#000000'
        x0, y0, x1, y1 = _pkd_[0][0] - _pkd_[0][2] - 3, _pkd_[0][1] - _pkd_[0][2] - 3, _pkd_[0][0] + _pkd_[0][2] + 3, _pkd_[0][1] + _pkd_[0][2] + 3
        for i in range(1, len(_pkd_)):
            x0, y0, x1, y1 = min(x0, _pkd_[i][0] - _pkd_[i][2] - 3), min(y0, _pkd_[i][1] - _pkd_[i][2] - 3), max(x1, _pkd_[i][0] + _pkd_[i][2] + 3), max(y1, _pkd_[i][1] + _pkd_[i][2] + 3)
        svg = [f'<svg x="0" y="0" width="{w}" height="{h}" viewBox="{x0} {y0} {x1-x0} {y1-y0}" xmlns="http://www.w3.org/2000/svg">']
        svg.append(f'<rect x="{x0}" y="{y0}" width="{x1-x0}" height="{y1-y0}" fill="#ffffff" />')
        svg.append(f'<circle cx="0.0" cy="0.0" r="0.25" fill="#ff0000" stroke="none" />')
        for i in range(len(_pkd_)):
            _c_ = _pkd_[i]
            svg.append(f'<circle cx="{_c_[0]}" cy="{_c_[1]}" r="{_c_[2]}" fill="none" stroke="{self.rt_self.co_mgr.getColor(i)}" stroke-width="0.2" />')
        if len(_chn_.keys()) > 0: 
            _index_ = list(_chn_.keys())[0]
            for i in range(len(_chn_.keys())+1):
                _index_next_ = _chn_[_index_]
                xy0, xy1 = _pkd_[_index_], _pkd_[_index_next_]
                svg.append(f'<line x1="{xy0[0]}" y1="{xy0[1]}" x2="{xy1[0]}" y2="{xy1[1]}" stroke="{_color_}" stroke-width="0.2" />')
                uv       = rt.unitVector((xy0, xy1))
                perp     = (-uv[1], uv[0])
                svg.append(f'<line x1="{xy1[0]}" y1="{xy1[1]}" x2="{xy1[0] - 1*uv[0] + 0.5*perp[0]}" y2="{xy1[1] - 1*uv[1] + 0.5*perp[1]}" stroke="{_color_}" stroke-width="0.1" />')
                svg.append(f'<line x1="{xy1[0]}" y1="{xy1[1]}" x2="{xy1[0] - 1*uv[0] - 0.5*perp[0]}" y2="{xy1[1] - 1*uv[1] - 0.5*perp[1]}" stroke="{_color_}" stroke-width="0.1" />')
                _index_  = _index_next_
        svg.append(self.rt_self.svgText(self.debug_str, x0, y1))
        svg.append('</svg>')
        return ''.join(svg)

    #
    # __packNextCircle__() - pack the next circle in this list
    #
    def __packNextCircle__(self):
        c = self.circles_left.pop(0)
        while self.nearest.queue[0][1] not in self.fwd.keys(): self.nearest.get() # find the next nearest circle that is still in the chain
        cm_i, cn_i    = self.nearest.queue[0][1], self.fwd[self.nearest.queue[0][1]]
        cm,   cn      = self.packed[cm_i], self.packed[cn_i]
        xy0, xy1      = self.rt_self.overlappingCirclesIntersections((cm[0], cm[1], cm[2] + c[2]), (cn[0], cn[1], cn[2] + c[2]))
        c             = (xy1[0], xy1[1], c[2])
        circle_placed = False
        while circle_placed == False:
            prev, next        = self.bck[cm_i], self.fwd[cn_i]
            seen              = set([cn_i, cm_i])
            overlapped_after  = None
            overlapped_before = None
            while overlapped_after is None and overlapped_before is None and next not in seen and prev not in seen:
                if self.rt_self.circlesOverlap((c[0], c[1], c[2]-self.epsilon), self.packed[next]): overlapped_after  = next
                if self.rt_self.circlesOverlap((c[0], c[1], c[2]-self.epsilon), self.packed[prev]): overlapped_before = prev
                seen.add(next), seen.add(prev)
                next, prev = self.fwd[next], self.bck[prev]
            #if overlapped_after  is not None and overlapped_before is not None:
            #   raise Exception()
            if   overlapped_after  is not None:
                self.__eraseChain__(cn_i, self.bck[overlapped_after])
                cn_i     = overlapped_after
                cn       = self.packed[cn_i]
                xy0, xy1 = self.rt_self.overlappingCirclesIntersections((cm[0], cm[1], cm[2] + c[2]), (cn[0], cn[1], cn[2] + c[2]))
                c        = (xy1[0], xy1[1], c[2])
            elif overlapped_before is not None:
                self.__eraseChain__(self.fwd[overlapped_before], cm_i)
                cm_i     = overlapped_before
                cm       = self.packed[cm_i]
                xy0, xy1 = self.rt_self.overlappingCirclesIntersections((cm[0], cm[1], cm[2] + c[2]), (cn[0], cn[1], cn[2] + c[2]))
                c        = (xy1[0], xy1[1], c[2])
            else:
                self.packed.append(c)
                _index_        = len(self.packed) - 1
                self.fwd[cm_i], self.bck[_index_] = _index_, cm_i
                self.bck[cn_i], self.fwd[_index_] = _index_, cn_i
                circle_placed = True
                self.nearest.put((c[0]**2 + c[1]**2, _index_))
        return

    #
    # __eraseChain__() - erase the chain from fm_i to to_i
    # ... inclusive -- fm_i and to_i will be deleted
    # ... at the end, the chain will be reconnected
    #
    def __eraseChain__(self, fm_i, to_i):
        i_start = self.bck[fm_i]
        i_end   = self.fwd[to_i]
        while fm_i != i_end:
            i_next = self.fwd[fm_i]
            del self.fwd[fm_i]
            fm_i   = i_next
        while to_i != i_start:
            i_prev = self.bck[to_i]
            del self.bck[to_i]
            to_i   = i_prev
        self.fwd[i_start], self.bck[i_end] = i_end, i_start
        # self.__validateChains__() # more for debug ... 
    
    #
    # __validateChains__()
    #
    def __validateChains__(self):
        if len(self.fwd) != len(self.bck):   raise Exception('Chains Are Different Lengths')
        for i in self.fwd.keys():
            if i not in self.bck.keys():     raise Exception('Forward Chain Has Key Not In Backward Chain')
            if self.bck[self.fwd[i]] != i:   raise Exception('Backward Chain Has Key Not In Forward Chain')

#
# 28s for 100_000 circles on M1 Pro (Unoptimized)
#
_tiles_ = []
for i in range(1):
    _circles_ = []
    for i in range(100_000): _circles_.append((0.0, 0.0, 0.8 + 1.0*random.random()))
    _cp_ = CirclePacker(rt, _circles_)
    _tiles_.append(_cp_)
rt.table(_tiles_, spacer=10, per_row=6)